In [ ]:
import pandas as pd
import numpy as np

timestamps = pd.read_csv("./data.csv").reset_index(drop=True)
display(timestamps[["kex", "auth"]].drop_duplicates())

,kex,auth
0,hqc-128,mldsa65-mldsa65-mlkem512
1023,ml-kem-512,mldsa65-mldsa65-mlkem512
2109,ml-kem-512,mldsa65-mldsa65-mldsa44
5512,hqc-128,mldsa65-mldsa65-hqc128


In [28]:
stats = timestamps.copy()
# compute durations and validate
stats["kex_dur"] = stats["sh_done"] - stats["ch_start"]
assert (stats["kex_dur"] > 0).all(), "non-positive key exchange duration"
stats["auth_dur"] = stats["auth_done"] - stats["auth_start"]
assert (stats["auth_dur"] > 0).all(), "non-positive authentication duration"
stats["hs_dur"] = stats["auth_done"] - stats["ch_start"]
assert (stats["hs_dur"] > 0).all(), "non-positive handshake duration"

# compute statistic
p90 = lambda x: np.percentile(x, 90)
p99 = lambda x: np.percentile(x, 99)
stats.groupby(["kex", "auth"])[
    ["kex_dur", "auth_dur", "hs_dur"]
    ].agg(
        ["median", "std"]
    ).reset_index().rename(columns={
        "<lambda_0>": "P90", "<lambda_1>": "P99"
    }).round(2).sort_values(("hs_dur", "median"))  

kex                      auth    kex_dur             auth_dur  \
                                            median        std    median   
2  ml-kem-512   mldsa65-mldsa65-mldsa44    31425.0   22132.27  278885.0   
3  ml-kem-512  mldsa65-mldsa65-mlkem512    29637.5  334155.95  249380.5   
1     hqc-128  mldsa65-mldsa65-mlkem512  1604524.0  268879.38  244680.0   
0     hqc-128    mldsa65-mldsa65-hqc128  1575690.0  213711.70  910301.0   

                 hs_dur             
         std     median        std  
2  107180.87   701122.0  245084.22  
3  452361.29   832025.0  668335.97  
1  334756.96  2384504.0  638951.89  
0  124104.30  3275863.0  562819.05